In [ ]:
import numpy as np
import graphinglib as gl
from copy import deepcopy

from src.hdu.grouped_maps import GroupedMaps
from src.hdu.map import Map
from src.coordinates.fits_coords import FitsCoords

In [ ]:
gm = GroupedMaps.load_from_loki(
    "data/loki/output_NGC4696_G235H_F170LP_full_OQBr_tied/NGC4696_G235H_F170LP_full_OQBr_tied_parameter_maps.fits"
)
SNR_cut = 3
arrows = [
    gl.Arrow((7.8+20, 2-23), (12.8+20, 7-23), "black"),
    gl.Arrow((8.2+20, 2-23), (3.2+20, 7-23), "black"),
    gl.Text(13+20, 7-23, "N", color="black", font_size=15, h_align="center", v_align="bottom"),
    gl.Text(3+20, 7-23, "E", color="black", font_size=15, h_align="center", v_align="bottom"),
]
AGN_pos = gl.Point(40.3, 0, marker_style="x", color="red", marker_size=50)

## Region spectra

In [ ]:
gm = GroupedMaps.load_from_loki("data/loki/output_NGC4696_G235H_F170LP_full_OQBr_tied/"
                                "NGC4696_G235H_F170LP_full_OQBr_tied_parameter_maps.fits")
gm["LINES.H210_S1.1.FLUX"].save("figures/results/region_spectra/H210_S1_flux.fits")
gm["LINES.H210_S1.TOTAL_SNR"].save("figures/results/region_spectra/H210_S1_snr.fits")

In [ ]:
flux = Map.load("figures/results/region_spectra/H210_S1_flux.fits")
snr = Map.load("figures/results/region_spectra/H210_S1_snr.fits")
flux = flux.mask(snr.data > 3)

import matplotlib.pyplot as plt

# Image
ny, nx = 57, 57

# Pixel edges
x = np.arange(nx)
y = np.arange(ny)
X, Y = np.meshgrid(x, y)

# Rotation angle
theta = np.deg2rad(-47)

# Rotation matrix
Xp = X*np.cos(theta) - Y*np.sin(theta)
Yp = X*np.sin(theta) + Y*np.cos(theta)

fig, ax = plt.subplots()

# Image (true pixel rotation)
ax.pcolormesh(Xp, Yp, flux.data, shading='nearest', cmap='cubehelix')

# Contours (use pixel centers!)
xc = np.arange(nx) + 0.5
yc = np.arange(ny) + 0.5
Xc, Yc = np.meshgrid(xc, yc)
Xcp = Xc*np.cos(theta) - Yc*np.sin(theta)
Ycp = Xc*np.sin(theta) + Yc*np.cos(theta)

ax.contour(Xcp, Ycp, img, colors='red')

ax.set_aspect('equal')
plt.show()

## Kinematics of the gas

In [ ]:
lines = [f"LINES.H210_{trans}.{p}" for trans in ["S3", "O3"] for p in ["1.FLUX", "1.VOFF", "1.FWHM"]]
hms = []

for i, line in enumerate(lines):
    map_ = gm[line]
    map_ = map_.mask(gm[f"{line[:14]}TOTAL_SNR"].data > SNR_cut)

    match i % 3:
        case 0:
            cmap_range = -18.2, -17.35
            cmap = "plasma"
        case 1:
            cmap_range = -250, 250
            cmap = "coolwarm"
        case 2:
            cmap = "viridis"
            cmap_range = 51, 240
            map_ /= 2 * np.sqrt(2 * np.log(2))  # FWHM to sigma

    hm = map_.rotate_field()
    hm.color_map = cmap
    hm.color_map_range = cmap_range
    if i > 2:
        hm.show_color_bar = False
    hms.append(hm)

hms[0].set_color_bar_params(position="top", label=r"\textbf{log($F$ [erg s$^{-1}$ cm$^{-2})$]}")
hms[1].set_color_bar_params(position="top", label=r"\textbf{velocity $v$ [km s$^{-1}$]}")
hms[2].set_color_bar_params(position="top", label=r"\textbf{vel. dispersion $\mathbf{\sigma}$ [km s$^{-1}$]}")

AGN_pos_white = gl.Point(40.3, 0, marker_style="x", color="white", marker_size=50)
white_arrows = deepcopy(arrows)
for arrow in white_arrows:
    arrow.color = "white"

texts = [gl.Text(58, 19, "H$_2$ 1-0 $S(3)$", h_align="right", v_align="top", color="black")]*3
texts += [gl.Text(58, 19, "H$_2$ 1-0 $O(3)$", h_align="right", v_align="top", color="black")]*3
text_boxes = [gl.Rectangle(43, 17, 14, 1, fill=False, line_width=10, edge_color="white")]*6

fig = gl.SmartFigure(
    2,
    3,
    # aspect_ratio=1,
    size=(6.4, 5.2),
    figure_style="dim",
    remove_x_ticks=True,
    remove_y_ticks=True,
    elements=[*list(zip(hms, text_boxes, texts, [AGN_pos_white]*6))],
    # elements=[*list(zip(hms, [AGN_pos]*6))],
    reference_labels_loc=(0.03, -0.16),
    width_padding=0,
    height_padding=0,
    height_ratios=[1.27, 1],
    x_lim=(20, 60),
    y_lim=(-23, 20),
).set_visual_params(use_latex=True, font_family="serif")#.show()
fig[1, 0] += white_arrows
fig.save("figures/meetings/january_2026/gas_kinematics.png", dpi=600, transparent=True)

## Kinematics of the stars

In [ ]:
lines = [f"LINES.H210_S1.{p}" for p in ["VPEAK", "1.FWHM"]]
lines += [f"CONTINUUM.STELLAR_{p}" for p in ["KINEMATICS.VEL", "KINEMATICS.VDISP"]]
hms = []

for i, line in enumerate(lines):
    map_ = gm[line]
    if i < 2:
        map_ = map_.mask(gm[f"{line[:14]}TOTAL_SNR"].data > SNR_cut)

    match i % 2:
        case 0: cmap = "coolwarm"
        case 1: cmap = "viridis"

    if i == 1:
        map_ /= 2 * np.sqrt(2 * np.log(2))  # FWHM to sigma

    hm = map_.rotate_field()
    hm.color_map = cmap
    hms.append(hm)

hms[0].set_color_bar_params(position="top", label=r"\textbf{velocity $v$ [km s$^{-1}$]}")
hms[1].set_color_bar_params(position="top", label=r"\textbf{vel. dispersion $\sigma$ [km s$^{-1}$]}")
hms[2].show_color_bar = False
hms[3].show_color_bar = False

hms[0].color_map_range = -65, 65
hms[2].color_map_range = -65, 65
hms[1].color_map_range = 87, 325
hms[3].color_map_range = 87, 325

texts = [gl.Text(58, 19, "H$_2$ 1-0 $S(1)$", h_align="right", v_align="top", color="black")]*2
texts += [gl.Text(58, 19, "Stars", h_align="right", v_align="top", color="black")]*2
text_boxes = [gl.Rectangle(45.5, 17.2, 11.5, 1, fill=False, line_width=10, edge_color="white")]*2
text_boxes += [gl.Rectangle(53, 17.5, 4, 1, fill=False, line_width=10, edge_color="white")]*2

fig = gl.SmartFigure(
    2,
    2,
    # aspect_ratio=1,
    size=(5, 6),
    figure_style="dim",
    remove_x_ticks=True,
    remove_y_ticks=True,
    elements=[*list(zip(hms, text_boxes, texts, [AGN_pos]*4))],
    reference_labels_loc=(0.03, -0.16),
    width_padding=0,
    height_padding=0,
    height_ratios=[1.24, 1],
    x_lim=(20, 60),
    y_lim=(-23, 20),
).set_visual_params(use_latex=True, font_family="serif")#.show()
fig[1, 0] += arrows
fig.save("figures/meetings/january_2026/stellar_kinematics.png", dpi=600, transparent=True)

## Approaching/receding components

In [ ]:
lines = [f"LINES.HI_PA_ALPHA.{i}.VOFF" for i in [1, 2]]

for i, line in enumerate(lines):
    map_ = gm[line]
    map_ = map_.mask(gm[f"{line[:20]}SNR"].data > SNR_cut)

    hm = map_.rotate_field()
    hm.color_map = "coolwarm"
    hms.append(hm)

hms[0].show_color_bar = False
hms[1].set_color_bar_params(position="right", label=r"\textbf{velocity $v$ [km s$^{-1}$]}")

hms[0].color_map_range = -800, 800
hms[1].color_map_range = -800, 800

texts = [gl.Text(58, 19, r"Pa$\alpha$", h_align="right", v_align="top", color="black")]*2
text_boxes = [gl.Rectangle(45.5, 17.2, 11.5, 1, fill=False, line_width=10, edge_color="white")]*2

fig = gl.SmartFigure(
    1,
    2,
    # aspect_ratio=1,
    size=(8, 5),
    figure_style="dim",
    remove_x_ticks=True,
    remove_y_ticks=True,
    # elements=[*list(zip(hms, text_boxes, texts, [AGN_pos]*2))],
    elements=[*list(zip(hms))],
    reference_labels_loc=(0.03, -0.16),
    width_padding=0,
    height_padding=0,
    x_lim=(20, 60),
    y_lim=(-23, 20),
).set_visual_params(use_latex=True, font_family="serif")#.show()
# fig[1, 0] += arrows
fig.save("figures/meetings/january_2026/approaching_receding_pa_alpha.png", dpi=600, transparent=True)

In [ ]:
stellar_velocity = gm["CONTINUUM.STELLAR_KINEMATICS.VEL"]
hm = stellar_velocity.data.plot
hm.set_color_bar_params(position="top", label=r"\textbf{stellar velocity $v$ [km s$^{-1}$]}")
hm.color_map = "coolwarm"
hm.color_map_range = -65, 65

fig = gl.SmartFigure(
    # aspect_ratio=1,
    size=(5, 6),
    remove_x_ticks=True,
    remove_y_ticks=True,
    elements=[hm],
    # x_lim=(20, 60),
    # y_lim=(-23, 20),
).set_visual_params(use_latex=True, font_family="serif").show()
fig.save("figures/meetings/january_2026/stellar_velocity_only.png", dpi=600)